In [1]:
import pandas as pd
from datetime import datetime, timedelta
from oodle.integrations.hooks.oodle_postgres_hook import OodlePostgresHook
from oodle import config
import os

In [2]:
def setup_connection():
    return OodlePostgresHook(config.REDSHIFT_SECRET).get_conn()

In [3]:
def return_fca_fields(date):
    q = """
WITH payment_holidays as (
select
distinct agreement_code,
        True as had_forbearance_this_week
from oodledata.forbearance
where status in ('Agreed', 'Expired')
and booking_status in ('Completed', 'Reviewed')
and type like '%Payment_Holiday%'
and created_date >= '{1}'
and created_date <= '{0}')
select sum(case when current_funding_entity in ('Oodle Funding Ltd', 'Dowson 2019-1 Plc', 'Dowson 2020-1 Plc') 
       then capital_balance end) as total_drawn_from_committed_facilities,
       count(distinct case when current_funding_entity 
       in ('Oodle Funding Ltd', 'Dowson 2019-1 Plc', 'Dowson 2020-1 Plc') then fe.agreement_code end) 
       as number_of_loans_committed_facility,
       sum(case when current_funding_entity in ('Oodle Funding Ltd', 'Dowson 2019-1 Plc', 'Dowson 2020-1 Plc')
           and holiday_adjusted_days_in_arrears > 0 then capital_balance end) as impaired_loans,
       sum(case when current_funding_entity in ('Oodle Funding Ltd', 'Dowson 2019-1 Plc', 'Dowson 2020-1 Plc')
           and holiday_adjusted_days_in_arrears > 0 and holiday_adjusted_days_in_arrears <= 365/12 
           then capital_balance end) as impaired_loans_less_than_1m,
       sum(case when current_funding_entity in ('Oodle Funding Ltd', 'Dowson 2019-1 Plc', 'Dowson 2020-1 Plc')
           and holiday_adjusted_days_in_arrears > 365/12 and holiday_adjusted_days_in_arrears <= 365/4 
           then capital_balance end) as impaired_loans_1m_3m,
       sum(case when current_funding_entity in ('Oodle Funding Ltd', 'Dowson 2019-1 Plc', 'Dowson 2020-1 Plc')
           and holiday_adjusted_days_in_arrears > 365/4 and holiday_adjusted_days_in_arrears <= 365 
           then capital_balance end) impaired_loans_3m_1y,
       sum(case when current_funding_entity in ('Oodle Funding Ltd', 'Dowson 2019-1 Plc', 'Dowson 2020-1 Plc')
           and holiday_adjusted_days_in_arrears > 365 then capital_balance end) as impaired_loans_more_than_1y,
       count(distinct case when current_funding_entity 
       in ('Oodle Funding Ltd', 'Dowson 2019-1 Plc', 'Dowson 2020-1 Plc')
           and holiday_adjusted_days_in_arrears > 0 then capital_balance end) as number_impaired_loans,
       count(distinct case when current_funding_entity 
       in ('Oodle Funding Ltd', 'Dowson 2019-1 Plc', 'Dowson 2020-1 Plc')
           and holiday_adjusted_days_in_arrears > 0 and holiday_adjusted_days_in_arrears <= 365/12 
           then lt.agreement_code end) as number_impaired_loans_less_than_1d,
       count(distinct case when current_funding_entity 
       in ('Oodle Funding Ltd', 'Dowson 2019-1 Plc', 'Dowson 2020-1 Plc')
           and holiday_adjusted_days_in_arrears > 365/12 and holiday_adjusted_days_in_arrears <= 365/4 
           then lt.agreement_code end) as number_impaired_loans_1m_3m,
       count(distinct case when current_funding_entity 
       in ('Oodle Funding Ltd', 'Dowson 2019-1 Plc', 'Dowson 2020-1 Plc')
           and holiday_adjusted_days_in_arrears > 365/4 and holiday_adjusted_days_in_arrears <= 365 
           then lt.agreement_code end) number_impaired_loans_3m_1y,
       count(distinct case when current_funding_entity 
       in ('Oodle Funding Ltd', 'Dowson 2019-1 Plc', 'Dowson 2020-1 Plc')
           and holiday_adjusted_days_in_arrears > 365 then capital_balance end) as number_impaired_loans_more_than_1y,
       sum(case when current_funding_entity in ('Oodle Funding Ltd', 'Dowson 2019-1 Plc', 'Dowson 2020-1 Plc')
           and holiday_adjusted_days_in_arrears = 0 
           and payment_holiday then capital_balance end) total_unimpaired_loans_ph,
       count(distinct case when current_funding_entity 
       in ('Oodle Funding Ltd', 'Dowson 2019-1 Plc', 'Dowson 2020-1 Plc') and holiday_adjusted_days_in_arrears = 0 
           and payment_holiday then lt.agreement_code end) number_total_unimpaired_loans_ph,
       sum(case when current_funding_entity in ('Oodle Funding Ltd', 'Dowson 2019-1 Plc', 'Dowson 2020-1 Plc')
           and holiday_adjusted_days_in_arrears > 0 and payment_holiday then capital_balance end) as impaired_loans_ph,
       sum(case when current_funding_entity in ('Oodle Funding Ltd', 'Dowson 2019-1 Plc', 'Dowson 2020-1 Plc')
           and holiday_adjusted_days_in_arrears > 0 and holiday_adjusted_days_in_arrears <= 365/12
           and payment_holiday then capital_balance end) as impaired_loans_less_than_1m_ph,
       sum(case when current_funding_entity in ('Oodle Funding Ltd', 'Dowson 2019-1 Plc', 'Dowson 2020-1 Plc')
           and holiday_adjusted_days_in_arrears > 365/12 and holiday_adjusted_days_in_arrears <= 365/4
           and payment_holiday then capital_balance end) as impaired_loans_1m_3m_ph,
       sum(case when current_funding_entity in ('Oodle Funding Ltd', 'Dowson 2019-1 Plc', 'Dowson 2020-1 Plc')
           and holiday_adjusted_days_in_arrears > 365/4 and holiday_adjusted_days_in_arrears <= 365
           and payment_holiday then capital_balance end) impaired_loans_3m_1y_ph,
       sum(case when current_funding_entity in ('Oodle Funding Ltd', 'Dowson 2019-1 Plc', 'Dowson 2020-1 Plc')
           and holiday_adjusted_days_in_arrears > 365
           and payment_holiday then capital_balance end) as impaired_loans_more_than_1y_ph,
       count(case when current_funding_entity in ('Oodle Funding Ltd', 'Dowson 2019-1 Plc', 'Dowson 2020-1 Plc')
           and holiday_adjusted_days_in_arrears > 0
           and payment_holiday then capital_balance end) as number_impaired_loans_ph,
       count(distinct case when current_funding_entity 
       in ('Oodle Funding Ltd', 'Dowson 2019-1 Plc', 'Dowson 2020-1 Plc')
           and holiday_adjusted_days_in_arrears > 0 and holiday_adjusted_days_in_arrears <= 365/12
           and payment_holiday then lt.agreement_code end) as number_impaired_loans_less_than_1m_ph,
       count(distinct case when current_funding_entity 
       in ('Oodle Funding Ltd', 'Dowson 2019-1 Plc', 'Dowson 2020-1 Plc')
           and holiday_adjusted_days_in_arrears > 365/12 and holiday_adjusted_days_in_arrears <= 365/4
           and payment_holiday then lt.agreement_code end) as number_impaired_loans_1m_3m_ph,
       count(case when current_funding_entity in ('Oodle Funding Ltd', 'Dowson 2019-1 Plc', 'Dowson 2020-1 Plc')
           and holiday_adjusted_days_in_arrears > 365/4 and holiday_adjusted_days_in_arrears <= 365
           and payment_holiday then lt.agreement_code end) number_impaired_loans_3m_1y_ph,
       count(case when current_funding_entity in ('Oodle Funding Ltd', 'Dowson 2019-1 Plc', 'Dowson 2020-1 Plc')
           and holiday_adjusted_days_in_arrears > 365
           and payment_holiday then capital_balance end) as number_impaired_loans_more_than_1y_ph,
       sum(case when current_funding_entity in ('Oodle Funding Ltd', 'Dowson 2019-1 Plc', 'Dowson 2020-1 Plc')
           and holiday_adjusted_days_in_arrears = 0
           and had_forbearance_this_week then capital_balance end) as sum_unimpaired_cb_newly_entering_ph,
       count(distinct case when current_funding_entity 
       in ('Oodle Funding Ltd', 'Dowson 2019-1 Plc', 'Dowson 2020-1 Plc')
           and holiday_adjusted_days_in_arrears = 0
           and had_forbearance_this_week then lt.agreement_code end) as number_unimpaired_newly_entering_ph,
       sum(case when current_funding_entity in ('Oodle Funding Ltd', 'Dowson 2019-1 Plc', 'Dowson 2020-1 Plc')
           and holiday_adjusted_days_in_arrears > 0
           and had_forbearance_this_week then capital_balance end) as sum_impaired_cb_newly_entering_ph,
       count(distinct case when current_funding_entity 
       in ('Oodle Funding Ltd', 'Dowson 2019-1 Plc', 'Dowson 2020-1 Plc')
           and holiday_adjusted_days_in_arrears > 0
           and had_forbearance_this_week
           then lt.agreement_code end) as number_impaired_newly_entering_ph
from oodledata_loans.loan_timeline lt
left join oodledata_loans.funding_entity fe
    on lt.agreement_code = fe.agreement_code
    and lt.date = fe.date
left join payment_holidays fb
    on lt.agreement_code = fb.agreement_code
where lt.date = '{0}'
and is_live;""".format(date.date(), date.date() - timedelta(7))
    q2 = """with q1 as (
select agreement_code,
       date,
       capital_balance,
       arrears,
       coalesce(days_since_last_payment_date::int,0) days_since_last_payment_date,
       lag(arrears_yesterday, coalesce(days_since_last_payment_date::int,0)) 
       over (partition by agreement_code order by date) lagged_arrears_before_pd,
       case when days_since_last_payment_date <= 7 then True
            else False end pd_in_last_7
from oodledata_loans.loan_timeline),
q2 as (select agreement_code,
       date,
       capital_balance,
       days_since_last_payment_date,
       lagged_arrears_before_pd,
       pd_in_last_7,
       case when pd_in_last_7 and arrears > lagged_arrears_before_pd then True
            else False end as missed_and_not_cured
from q1)
select sum(case when missed_and_not_cured then capital_balance end) as value_of_loans_missing,
       count(distinct case when missed_and_not_cured then agreement_code end) as number_of_loans_missing
from q2
where date = '{0}';""".format(date.date())
    q3 = """
    select sum(case when stage_name in ('Closed - Declined') then total_finance_amount end) sum_finance_declined,
       count(case when stage_name in ('Closed - Declined') then total_finance_amount end) number_declined
    from oodledata.applications
    where closed_date > '{1}'
    and closed_date <= '{0}'""".format(date.date(), date.date() - timedelta(7))
    df1 = pd.read_sql(q, setup_connection())
    df2 = pd.read_sql(q2, setup_connection())
    df5 = pd.read_sql(q3, setup_connection())
    df3 = pd.concat([df1, df2, df5], axis=1)
    df4 = df3[[
        'total_drawn_from_committed_facilities',
        'impaired_loans',
        'total_unimpaired_loans_ph',
        'impaired_loans_ph',
        'number_of_loans_committed_facility',
        'number_impaired_loans',
        'number_total_unimpaired_loans_ph',
        'number_impaired_loans_ph',
        'value_of_loans_missing',
        'sum_unimpaired_cb_newly_entering_ph',
        'sum_impaired_cb_newly_entering_ph',
        'number_of_loans_missing',
        'number_unimpaired_newly_entering_ph',
        'number_impaired_newly_entering_ph',
        'impaired_loans_less_than_1m',
        'number_impaired_loans_less_than_1d',
        'impaired_loans_1m_3m',
        'number_impaired_loans_1m_3m',
        'impaired_loans_3m_1y',
        'number_impaired_loans_3m_1y',
        'impaired_loans_more_than_1y',
        'number_impaired_loans_more_than_1y',
        'impaired_loans_less_than_1m_ph',
        'number_impaired_loans_less_than_1m_ph',
        'impaired_loans_1m_3m_ph',
        'number_impaired_loans_1m_3m_ph',
        'impaired_loans_3m_1y_ph',
        'number_impaired_loans_3m_1y_ph',
        'impaired_loans_more_than_1y_ph',
        'number_impaired_loans_more_than_1y_ph',
        'sum_finance_declined',
        'number_declined']]
    
    df4.T.to_excel('/home/admin/local/FCA_weekly_numbers_{}.xlsx'.format(date.date()))
    return df4.T
if __name__ == "__main__":
    fca_report = return_fca_fields(datetime(2020, 8, 28))